# Import all the necessary libraries

In [ ]:
import cv2, time, pandas
from datetime import datetime

# Initialize the variables

In [ ]:
first = None # This variable holds the value of the first frame
status_list = [None,None] # This variable holds the list of statuses - if Python has come across a frame greater than 1000 pixels
times = [] # This variable holds the timestamps during which a motion that was detected started and ended
df = pandas.DataFrame(columns = ["Start","End"]) # The dataframe that will hold the timestamp for each detected motion

# Capture the video

In [ ]:
vid = cv2.VideoCapture(0) # To capture the video from the first camera of the device

# Process the video

In [ ]:
while True:
    
    check, frame = vid.read() # To read frame by frame
    status = 0
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)# To convert the image to grayscale
    gray_frame = cv2.GaussianBlur(gray_frame,(21,21),0) # To make the image blurry so that noise is reduced and accuracy is improved
    
    if first is None:
        first = gray_frame
        continue
    
    
    diff = cv2.absdiff(first, gray_frame) # To compute and store the absolute difference between the frames
    thresh_diff = cv2.threshold(diff,30,255,cv2.THRESH_BINARY)[1] # To convert differences less than 30 as WHITE and more than 30 as BLACK
    thresh_diff = cv2.dilate(thresh_diff,None,iterations = 2) # To increase the object area
    (_,cnts,_) = cv2.findContours(thresh_diff.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) # To find the contours of the frame
    
    
    for contour in cnts: # To iterate through the contours
        if cv2.contourArea(contour) < 1000: # To check if the contour area is less than 1000 pixels
            continue
        status = 1 # To remember that Python has found a frame that is bigger than 1000px
        (x,y,w,h)= cv2.boundingRect(contour) # To find the rectangle bounding the contours
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3) # To draw a rectangle around the object
        
    status_list.append(status)
    if status_list[-1] == 1 and status_list[-2] == 0: # To figure out the start time
        times.append(datetime.now()) # To store the start time
    if status_list[-1] == 0 and status_list[-2] == 1: # To figure out the end tome
        times.append(datetime.now()) # To store the end time
        
        
    cv2.imshow("Gray",gray_frame) # To display the frames that are captured in gray scale
    cv2.imshow("Difference",diff) # To display the difference between the frames
    cv2.imshow("Threshold",thresh_diff) # To display the difference after the threshold has been applied
    cv2.imshow("Color",frame) # To display the original color frames
    
    key = cv2.waitKey(1) # To wait till a key is pressed
    
    if key == ord('q'): # To quit if the key pressed is q
        if status == 1:
            times.append(datetime.now())
        break
        
           

# Write the timestamps to a csv file

In [ ]:
for i in range(0,len(times),2):
    df=df.append({"Start":times[i],"End":times[i+1]},ignore_index=True) # To store the start and end times of when each motion was detected

df.to_csv("Times.csv") # To write to the Times.csv file

# End the video processing

In [ ]:
vid.release() # To stop the capturing device from capturing video
cv2.destroyAllWindows # To close the windows and deallocate associated memory usage